<a href="https://colab.research.google.com/github/ffatmanurggultekin/scoliosis-detection/blob/main/densenet201_imadjust_sobel_diff_alpha2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#############################################
# 1) Klasör Yolları ve Ayarlar
#############################################
train_dir = "/content/drive/MyDrive/train_imadjust_sobel_diff_alpha2.0"
val_dir = "/content/drive/MyDrive/validation_imadjust_sobel_diff_alpha2.0"

batch_size = 32
epochs = 50
n_splits = 5  # Cross-validation kat sayısı


In [ ]:
#############################################
# 2) Veri Yollarını ve Etiketlerini Toplama
#############################################
def gather_paths_and_labels(dir_path):
    filepaths = []
    labels = []

    normal_path = os.path.join(dir_path, "Normal")
    scol_path = os.path.join(dir_path, "Scol")

    if os.path.exists(normal_path):
        for fname in os.listdir(normal_path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                filepaths.append(os.path.join(normal_path, fname))
                labels.append(0)  # Normal => 0

    if os.path.exists(scol_path):
        for fname in os.listdir(scol_path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                filepaths.append(os.path.join(scol_path, fname))
                labels.append(1)  # Scol => 1

    return pd.DataFrame({'filepath': filepaths, 'label': labels})

# Train ve Validation'dan gelen DataFrame'leri birleştirelim
df_train = gather_paths_and_labels(train_dir)
df_val = gather_paths_and_labels(val_dir)
df_trainVal = pd.concat([df_train, df_val], ignore_index=True)

# Label sütununu string formatına çevir
df_trainVal['label'] = df_trainVal['label'].astype(str)

print(f"Train+Validation birleşik boyutu: {len(df_trainVal)}")


Train+Validation birleşik boyutu: 214


In [ ]:
#############################################
# 3) DenseNet201 Modelini Oluşturma
#############################################
def create_densenet201_model():
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False  # Fine-tuning yapmayacaksanız dondurun

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=base_model.input, outputs=output)
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model



In [ ]:
#############################################
# 4) Cross-Validation Fonksiyonu
#############################################
def run_5fold_cv(df, n_splits=5, epochs=50, batch_size=32):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    X = df['filepath'].values
    y = df['label'].values

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    fold_no = 1

    # Data Augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    val_datagen = ImageDataGenerator(rescale=1./255)

    for train_idx, val_idx in skf.split(X, y):
        print(f"\n=== Fold {fold_no}/{n_splits} ===")

        # Fold için Train/Validation veri setlerini ayır
        train_fold = df.iloc[train_idx]
        val_fold = df.iloc[val_idx]

        # Data generator'lar
        train_gen = train_datagen.flow_from_dataframe(
            train_fold,
            x_col='filepath',
            y_col='label',
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='binary'
        )
        val_gen = val_datagen.flow_from_dataframe(
            val_fold,
            x_col='filepath',
            y_col='label',
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='binary'
        )

        # Modeli oluştur
        model = create_densenet201_model()

        # EarlyStopping
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

        steps_per_epoch = math.ceil(len(train_fold) / batch_size)
        validation_steps = math.ceil(len(val_fold) / batch_size)

        # Modeli eğit
        history = model.fit(
            train_gen,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=val_gen,
            validation_steps=validation_steps,
            callbacks=[early_stopping],
            verbose=1
        )

        # Validation setinde değerlendirme
        val_loss, val_acc = model.evaluate(val_gen, steps=validation_steps, verbose=0)
        accuracy_list.append(val_acc)

        # Confusion Matrix ve Classification Report
        y_val_pred_prob = model.predict(val_gen, steps=validation_steps, verbose=0)
        y_val_pred = (y_val_pred_prob > 0.5).astype(int).ravel()
        y_val_true = val_fold['label'].astype(int).values

        precision = precision_score(y_val_true, y_val_pred, average='binary')
        recall = recall_score(y_val_true, y_val_pred, average='binary')
        f1 = f1_score(y_val_true, y_val_pred, average='binary')

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print("\nValidation Metrics:")
        print(f"Accuracy: {val_acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        fold_no += 1

    mean_acc = np.mean(accuracy_list)
    mean_precision = np.mean(precision_list)
    mean_recall = np.mean(recall_list)
    mean_f1 = np.mean(f1_list)

    print("\n5-Fold CV Metrics:")
    print(f"Mean Accuracy: {mean_acc:.4f}")
    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"Mean F1-Score: {mean_f1:.4f}")

    return mean_acc, mean_precision, mean_recall, mean_f1


In [ ]:
#############################################
# 5) 5-Fold Cross-Validation'ı Çalıştır
#############################################
mean_acc, mean_precision, mean_recall, mean_f1 = run_5fold_cv(df_trainVal, n_splits=n_splits, epochs=epochs, batch_size=batch_size)
print(f"\nFinal Results:\nAccuracy = {mean_acc:.4f}, Precision = {mean_precision:.4f}, Recall = {mean_recall:.4f}, F1-Score = {mean_f1:.4f}")


=== Fold 1/5 ===
Found 171 validated image filenames belonging to 2 classes.
Found 43 validated image filenames belonging to 2 classes.
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 118s 11s/step - accuracy: 0.3868 - loss: 0.9170 - val_accuracy: 0.7209 - val_loss: 0.5444
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.7126 - loss: 0.5985 - val_accuracy: 0.7209 - val_loss: 0.6051
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.7222 - loss: 0.6250 - val_accuracy: 0.7209 - val_loss: 0.5672
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 150ms/step - accuracy: 0.6761 - loss: 0.7457 - val_accuracy: 0.7209 - val_loss: 0.4718
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 143ms/step - accuracy: 0.6756 - loss: 0.6051 - val_accuracy: 0.7442 - val_loss: 0.4285
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.7488 - loss: 0.4486 - val_accuracy: 0.7674 - val_loss: 0.4148
Epoch 12/50
6/6 ━━━━━

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.6743 - loss: 0.6801 - val_accuracy: 0.7209 - val_loss: 0.6000
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 147ms/step - accuracy: 0.6921 - loss: 0.6638 - val_accuracy: 0.7209 - val_loss: 0.5761
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 158ms/step - accuracy: 0.6837 - loss: 0.5648 - val_accuracy: 0.7209 - val_loss: 0.5308
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step - accuracy: 0.7315 - loss: 0.5329 - val_accuracy: 0.7209 - val_loss: 0.5093
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - accuracy: 0.7253 - loss: 0.4815 - val_accuracy: 0.7209 - val_loss: 0.5008
Epoch 12/50
6/6 ━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 84s 10s/step - accuracy: 0.4474 - loss: 0.9859 - val_accuracy: 0.7619 - val_loss: 0.5553
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.7005 - loss: 0.6213 - val_accuracy: 0.7381 - val_loss: 0.5447
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 150ms/step - accuracy: 0.7088 - loss: 0.7218 - val_accuracy: 0.7381 - val_loss: 0.4898
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.7179 - loss: 0.6154 - val_accuracy: 0.7619 - val_loss: 0.4644
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.7417 - loss: 0.4754 - val_accuracy: 0.7381 - val_loss: 0.4774
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.6883 - loss: 0.5373 - val_accuracy: 0.7619 - val_loss: 0.4541
Epoch 12/50
6/6 ━━━━